#### 搭建PyTorch神经网络进行气温预测

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
features = pd.read_csv("temps.csv")

#看看数据是什么样的
features.head()

FileNotFoundError: [Errno 2] No such file or directory: 'temps.csv'

#### 数据表中
- year, month, day,week 分别表示具体的时间
- temp_2:前天的最高气温
- temp_1:昨天的最高气温
- average:在历史中，每年这一天的平均最高气温
- actual：这就是我们的标签值，当天的真实最高气温
- friend:这一列是凑热闹的，你的朋友猜测的可能值，咱们不管它就好了

In [4]:
print("数据维度:", features.shape)

NameError: name 'features' is not defined

In [5]:
# 处理时间数据
import datetime

# 分别得到 年月日
years = features['year']
months = features['month']
days = features['day']

# datetime格式
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
dates[:5]

NameError: name 'features' is not defined

In [9]:
# 准备画图
# 指定默认风格
plt.style.use('fivethiryeight')

# 设置布局
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig.autofmt_xdate(rotation=45)

#标签值
ax1.plot(dates, features['actual'])
ax1.set_xlabel('')
ax1.set_ylabel('Temperature')
ax1.set_title('Max Temp')

# 昨天
ax2.plot(dates, features['temp_1'])
ax2.set_xlabel('')
ax2.set_ylabel('Temperature')
ax2.set_title('Max Temp')

# 前天
ax3.plot(dates, features['temp_2'])
ax3.set_xlabel('')
ax3.set_ylabel('Temperature')
ax3.set_title('Max Temp')

# 我的朋友
ax4.plot(dates, features['temp_2'])
ax4.set_xlabel('')
ax4.set_ylabel('Temperature')
ax4.set_title('Max Temp')

plt.tight_layout(pad=2)


OSError: 'fivethiryeight' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [10]:
features = pd.get_dummies(features)
features.head(5)

NameError: name 'features' is not defined

In [11]:
# 标签
labels = np.array(features['acutal'])
# 在特征中去掉标签
features = features.drop('acutal', axis = 1)
# 名字去掉单独保存一下
features_list = list(features.colums)

#转换成合适的格式
features = np.array(features)

NameError: name 'features' is not defined

In [12]:
features.shape

NameError: name 'features' is not defined

In [13]:
from sklearn import preprocessing
input_features = preprocessing.StandardScaler().fit_transform(features)

NameError: name 'features' is not defined

In [14]:
input_features[0]

NameError: name 'input_features' is not defined

#### 构建网络模型

In [ ]:
x = torch.tensor(input_features, dtype = float)
y = torch.tensor(labels, dtype = float)

# 权重参数初始化
weights = torch.randn((14, 128), dtype = float, requires_grad = True)
biases = torch.randn(128, dtype = float, requires_grad = True)
weights2 = torch.randn((128, 1), dtype = float, requires_grad = True)
biases2 = torch.randn(1, dtype = float, requires_grad = True)

learning_rate = 0.001
losses = []

for i in range(1000):
    # 计算隐层
    hidden = x.mm(weights) + biases
    # 加入激活函数
    predictions = hidden.mm(weights2) + biases2
    # 通过计算损失
    loss = torch.mean((predictions - y) ** 2)
    losses.append(loss.data.numpy())

    # 打印损失值
    if i % 100 == 0:
        print('loss', loss)
    # 反向传播计算
    loss.backward()

    # 更新参数
